### CONDA ENVIRONMENTS

For steps __1. preprocess__ and __2. mip__, `conda activate g5ht-pipeline`

For step __3. segment__, `conda activate segment-torch` or `conda activate torchcu129`

For step __4. spline, 5. orient, 6. warp, 7. reg__

In [ ]:
import sys
import os
import importlib

import utils

In [ ]:
DATA_PTH = r'D:\\DATA\\g5ht-free\\20251028\\'

# SPECIFY DATA TO PROCESS

In [ ]:
DATA_PTH = r'D:\\DATA\\g5ht-free\\20251028\\'
INPUT_ND2 = 'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)

NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\noise_042925.tif'

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 41

noise_stack = utils.get_noise_stack(NOISE_PTH, STACK_LENGTH)

num_frames, height, width, num_channels = utils.get_range_from_nd2(INPUT_ND2_PTH, stack_length=STACK_LENGTH) 


print(INPUT_ND2)
print('Num z-slices: ', STACK_LENGTH)
print('Number of frames: ', num_frames)
print('Height: ', height)
print('width: ', width)
print('Number of channels: ', num_channels)

# 1. PREPROCESS

### PARALLEL

if processing locally, would recommend setting the 6th arg (number of workers) to something less than 10 if you want to use your computer at the same time

In [ ]:
from preprocess_parallel import main as preprocess_nd2_parallel
_ = importlib.reload(sys.modules['preprocess_parallel'])

# command-line arguments
# sys.argv = ["", INPUT_ND2_PTH, "0", str(num_frames), NOISE_PTH, STACK_LENGTH, 5]
sys.argv = ["", INPUT_ND2_PTH, "0", str(num_frames-1), NOISE_PTH, STACK_LENGTH, 5, num_frames, height, width, num_channels]

# Call the main function
preprocess_nd2_parallel()

### SERIAL

In [ ]:
from preprocess import main as preprocess_nd2
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
# sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH]
sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH, STACK_LENGTH, num_frames, height, width, num_channels]

# Call the main function
preprocess_nd2()

### TESTING CHANGES

In [44]:
import preprocess_testing
_ = importlib.reload(sys.modules['utils'])
_ = importlib.reload(sys.modules['preprocess_testing'])

# command-line arguments
# sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH]
sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH, STACK_LENGTH, num_frames, height, width, num_channels]

# Call the main function
preprocess_testing.main()

# 2. MAX INTENSITY PROJECTION

In [ ]:
from mip import main as mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
sys.argv = ["", INPUT_ND2_PTH, STACK_LENGTH, num_frames, 30]

# Call the main function
mip()

# 3. SEGMENT

### on home pc
`conda activate segment-torch`

Uses a separate conda environment from the rest of the pipeline. create it using:
`conda env create -f segment_torch.yml`

### on lab pc
`conda activate torchcu129`

Uses a separate conda environment from the rest of the pipeline. create it following steps in:
`segment_torch_cu129_environment.yml`

### setup each time model weights change
Need to set path to model weights as `CHECKPOINT` in `eval_torch.py`

In [ ]:

import sys
import os

DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121\date-20251121_strain-ISg5HT_condition-fedpatch_worm002'
INPUT_ND2 = 'date-20251121_strain-ISg5HT_condition-fedpatch_worm002.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)
OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]
MIP_PTH = os.path.join(DATA_PTH, 'mip.tif')

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 15

print(INPUT_ND2)


In [ ]:
from segment.segment_torch import main as segment_mip


# command-line arguments
sys.argv = ["", MIP_PTH]

segment_mip()


# 4. SPLINE

`conda activate g5ht-pipeline`

In [ ]:
import sys
import os
from spline import main as get_spline

# PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001\label.tif'
PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121\date-20251121_strain-ISg5HT_condition-fedpatch_worm002\label.tif'

# command-line arguments
sys.argv = ["", PTH]

get_spline()

# 5. ORIENT

`conda activate g5ht-pipeline`

In [ ]:
import sys
import os
from orient import main as find_orientation

# PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001\spline.json'
PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121\date-20251121_strain-ISg5HT_condition-fedpatch_worm002\spline.json'

# command-line arguments
sys.argv = ["", PTH, 108, 42]

find_orientation()

# 6. WARP

`conda activate g5ht-pipeline`

TODO: parallelize

In [ ]:
import sys
import os
from warp import main as warp_worm
from tqdm import tqdm
from utils import get_range_from_nd2

DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121'
INPUT_ND2 = 'date-20251121_strain-ISg5HT_condition-fedpatch_worm002.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)
STACK_LENGTH = 15
num_frames, height, width, num_channels = get_range_from_nd2(INPUT_ND2_PTH, stack_length=STACK_LENGTH) 

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121\date-20251121_strain-ISg5HT_condition-fedpatch_worm002'

for i in tqdm(range(num_frames)):
    # command-line arguments
    sys.argv = ["", PTH, i]

    warp_worm()

### TEST PARALLEL (TODO)

In [ ]:
from warp_parallel import main as warp_worm_parallel

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'

# command-line arguments
sys.argv = ["", PTH, "0", "1199"]

# Call the main function
warp_worm_parallel()

# 7. REG

`conda activate g5ht-pipeline`

TODO: parallelize

In [1]:
import sys
import os
from tqdm import tqdm
import importlib

from reg import main as reg_worm

PTH = r'D:\\DATA\\g5ht-free\\20251028\\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'


for i in tqdm(range(117,1200)):
    # command-line arguments
    sys.argv = ["", PTH, i, "3"]
    reg_worm()

  7%|▋         | 77/1083 [2:53:46<37:50:16, 135.40s/it]


KeyboardInterrupt: 

### View fixed and move as one image (see reg_microfilm.ipynb)

# 8. QUANTIFY

`conda activate g5ht-pipeline`

Have to first label dorsal and ventral nerve rings and pharynx. See ...

In [ ]:
import sys
import os
from quantify import main as quantify_roi_intensity
from numpy import genfromtxt
import matplotlib.pyplot as plt
import numpy as np

import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 15}
matplotlib.rc('font', **font)

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'


sys.argv = ["", PTH]
quantify_roi_intensity()

In [ ]:
roi_intensity = genfromtxt(os.path.join(PTH,'quantified.csv'), delimiter=',')
roi_intensity = roi_intensity[1:,:]
roi_intensity

In [ ]:
%matplotlib qt

t = roi_intensity[:,0]

plt.figure(figsize=(10, 4))
plt.plot(t, roi_intensity[:, 1] / np.mean(roi_intensity[:60, 1]), label='Dorsal nerve ring', lw=3)
plt.plot(t, roi_intensity[:, 2] / np.mean(roi_intensity[:60, 2]), label='Ventral nerve ring', lw=3)
plt.plot(t, roi_intensity[:, 3] / np.mean(roi_intensity[:60, 3]), label='Pharynx', lw=3)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
plt.xlabel('Time (min)')
plt.ylabel(r'$F/F_{baseline}$')
plt.xlim(t[0],t[-1])
plt.axhline(1, ls='--', c='k', zorder=0)
plt.tight_layout()
plt.show()
